In [1]:
import pandas as pd
import datetime as dt
import pymysql
import matplotlib.pyplot as plt
from matplotlib.lines import TICKLEFT, TICKRIGHT, Line2D
from matplotlib.patches import Rectangle
import matplotlib as mpl
import matplotlib.dates as mdate
from datetime import time

class Kline(object):
    def DrawKline(self,aa):
    #     date='2014-01-03'
        # aa=df1[str(date)]
        aa.reset_index(drop=True,inplace=True)
        fig =plt.figure(figsize=(30, 12),facecolor='w')

        #副图是主图的比例
        w1=0.25
        w2=0.8/(1+w1)*w1
        rec1=[0.1,0.1+w2,0.9,w2/w1]
        rec2=[0.1,0.1,0.9,w2]
        ax=fig.add_axes(rec1)
        ax1=fig.add_axes(rec2,sharex=ax)
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax1.get_xticklabels(), visible=True)

        #设置时间刻度
        dlist=aa.datetime.apply(lambda x:str(x.time()))
        dd=dlist.values
        d1=[i*100 for i in range(len(dd))]
        ax1.set_xticks(d1[::30])
        ax1.set_xticklabels(dd[::30])

        Kline={}
        Kline['W'] = 80
        Kline['Diff'] = 20
        Kline['WW']=100
        d=0
        rows=len(aa)
        for i in aa.iterrows():
            Row=i[1]
            x1 =(d+1)*Kline['WW']
            x2 = x1-Kline['W']/2
            O = Row['open']
            C = Row['close']
            L = Row['low']
            H = Row['high']
            hh = abs(C - O)

            if C > O:
                a = False
                y = O
                cc = 'R'
                # 上影线 下影线
                line1 = Line2D((x1, x1), (C, H), color=cc)
                line2 = Line2D((x1, x1), (O, L), color=cc)

                ax.add_line(line1)
                ax.add_line(line2)
            else:
                a = True
                y = C
                cc = 'C'
                # 影线
                line = Line2D((x1, x1), (L, H), color=cc)
                ax.add_line(line)
            hh = abs(C - O)

            # K线实体
            rec = Rectangle((x2, y), Kline['W'], hh, fill=a, color=cc)
            ax.add_patch(rec)

            if Row['macd']>0:
                rec= Rectangle((x2, 0), Kline['W'], Row['macd'], fill=a, color='R')
            else:
                rec= Rectangle((x2, 0), Kline['W'], Row['macd'], fill=a, color='C')
            ax1.add_patch(rec)
            d+=1
        if "ma60" in aa.columns:
            ax.plot(aa.index*Kline['WW']+Kline['WW'],aa['ma60'],c='r')
        if "ma30" in aa.columns:
            ax.plot(aa.index*Kline['WW']+Kline['WW'],aa['ma30'],c='b')
        ax1.axhline(0,linestyle="--",linewidth=0.5,color='k')
        if "diff" in aa.columns:
            ax1.plot(aa.index*Kline['WW']+Kline['WW'],aa['diff'],c='b',linewidth=0.5)
        if "dea" in aa.columns:
            ax1.plot(aa.index*Kline['WW']+Kline['WW'],aa['dea'],c='y',linewidth=0.5)

        ax.grid()
        ax1.grid()
        ax.autoscale(tight=False)
        ax.autoscale_view()

        time1=aa.iloc[0]['datetime']
        time2=aa.iloc[-1]['datetime']
        title="%s--%s" %(time1,time2)
        ax.set_title(title, fontsize=20, color='r')
        # fig.savefig("a.jpg")
        plt.show()
        fig.clear()
        plt.close(fig)

print("OK")


OK


In [3]:
aa=pd.read_pickle("pkl/HSI1903")
aa[aa.trade_date=='2019-03-28']

,datetime,code,open,high,low,close,trade_date,macd,diff,dea,ma30,ma60,chg,std60,std1
datetime,,,,,,,,,,,,,,,
2019-03-28 09:15:00,2019-03-28 09:15:00,HSI1903,28567.0,28599.0,28557.0,28594.0,2019-03-28,-21.662757,-10.669442,-10.993315,28712.423105,28727.338884,27.0,7.422863,3.637411
2019-03-28 09:16:00,2019-03-28 09:16:00,HSI1903,28593.0,28606.0,28587.0,28596.0,2019-03-28,-27.347529,-14.005059,-13.342469,28704.911937,28723.032691,3.0,7.371793,0.406957
2019-03-28 09:17:00,2019-03-28 09:17:00,HSI1903,28598.0,28604.0,28596.0,28602.0,2019-03-28,-28.790082,-16.962064,-11.828018,28698.272457,28719.064406,4.0,7.389839,0.541284
2019-03-28 09:18:00,2019-03-28 09:18:00,HSI1903,28603.0,28603.0,28586.0,28588.0,2019-03-28,-30.242610,-19.618173,-10.624437,28691.158105,28714.767213,-15.0,7.591003,-1.976023
2019-03-28 09:19:00,2019-03-28 09:19:00,HSI1903,28590.0,28617.0,28589.0,28616.0,2019-03-28,-26.098859,-20.914310,-5.184549,28686.309195,28711.528943,26.0,8.323644,3.123632
2019-03-28 09:20:00,2019-03-28 09:20:00,HSI1903,28615.0,28615.0,28591.0,28595.0,2019-03-28,-25.216422,-21.774733,-3.441689,28680.418279,28707.708322,-20.0,8.491564,-2.355279
2019-03-28 09:21:00,2019-03-28 09:21:00,HSI1903,28594.0,28599.0,28590.0,28595.0,2019-03-28,-23.574667,-22.134719,-1.439947,28674.907423,28704.012968,1.0,8.488162,0.117811
2019-03-28 09:22:00,2019-03-28 09:22:00,HSI1903,28593.0,28614.0,28593.0,28599.0,2019-03-28,-21.018676,-21.911511,0.892835,28670.010169,28700.569919,6.0,8.517612,0.704423
2019-03-28 09:23:00,2019-03-28 09:23:00,HSI1903,28605.0,28605.0,28593.0,28593.0,2019-03-28,-19.341157,-21.397440,2.056283,28665.041771,28697.043037,-12.0,8.644202,-1.388214


In [43]:
def abc(*a,**b):
    print(a)
    print(b)

In [53]:
ll=[1,2,4]
cc={1:1,2:2,4:4}
abc(ll,**cc)

TypeError: abc() keywords must be strings